In [1]:
import numpy as np
import pandas as pd
import sys
import os
import csv
import ipdb
import pickle
from collections import OrderedDict, defaultdict
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [9]:
CONFIG = {
    'week': 'week12',
    'current_week': 12,
    'thresh': 0.5
}
policies = ['rmab', 'control', 'round_robin']
T = 12

df = pd.read_csv('outputs/analysis_lists/all_analysis_week_12.csv')


In [3]:
## Get users in round robin, rmab and control groups

df2 = pd.read_csv('outputs/individual_clustering/weekly_kmeans_pilot_stats_40.csv')


rmab_list = pd.read_csv('outputs/pilot_outputs/rmab_pilot.csv')['user_id'].to_list()
round_robin_list = pd.read_csv('outputs/pilot_outputs/round_robin_pilot.csv')['user_id'].to_list()
control_list = pd.read_csv('outputs/pilot_outputs/control_pilot.csv')['user_id'].to_list()


rmab_group = df[df['user_id'].isin(rmab_list)]
rmab_group = rmab_group.sort_values('{}_whittle'.format(CONFIG['week']), ascending=False)

round_robin_group = df2[df2['user_id'].isin(round_robin_list)]
round_robin_group = round_robin_group.sort_values('registration_date', ascending=True)

rmab_user_ids = rmab_group['user_id'].to_list()
round_robin_user_ids = round_robin_group['user_id'].to_list()

In [6]:
## Load pilot registration data
pilot_pd_data = pd.read_csv("feb16-mar15_data/beneficiary/ai_registration-20210216-20210315.csv", sep='\t')
class_based_features = ['ChannelType', 'phone_owner' , "call_slots", "language"]
numeric_features = ['enroll_gest_age', "enroll_delivery_status", 
                    'age_binned', 'g', 'p', 's', 'l', 'a', 'education', 'income_binned']

# Bin income and age
pilot_pd_data['income_binned'] = pilot_pd_data['income_bracket'].replace(
    dict(zip(['0-5000', '5000-10000', '10000-15000', '15000-20000',
        '20000-25000', '25000-30000', '30000 and above'], [0,1,2,3,4,5,6])))
pilot_pd_data['age_binned'] = pd.cut(pilot_pd_data['age'], bins=5, labels=False)


dummy = pd.get_dummies(pilot_pd_data[class_based_features], columns = class_based_features)
pilot_pd_data = pd.concat([pilot_pd_data.drop(columns = class_based_features), dummy], axis=1)
class_based_features = list(dummy.columns)




In [7]:
# Define columns
columns = numeric_features+class_based_features +["curr_state"] +["exp_arm_rmab","exp_arm_rr"]

# Reorder and subset pilot_pd_data same as df user_ids
pilot_user_ids = df.user_id
pilot_static_features = pilot_pd_data[pilot_pd_data['user_id'].isin(pilot_user_ids)][numeric_features+class_based_features+['user_id']]
pilot_static_features = pilot_static_features.set_index('user_id')
pilot_static_features = pilot_static_features.loc[pilot_user_ids].values


In [ ]:
# X matrix contains features
# Y_mat contains for every time, cumulative engagement upto time T.
##   Here weekly engagement is defined as binary - listened to at least one call
# Y_all_mat contains cumulative calls listened to. No binarization
# C_mat contains cumulative number of calls made upto time t.

X_mat, Y_mat, Y_all_mat, C_mat = [], [[] for i in range(1,T)], [[] for i in range(1,T)], [[] for i in range(1,T)]

intervention_benefit = {'rmab': [], 'round_robin': [], 'control': []}
all_user_ids = set(df['user_id'].to_list())
full_mat = {'rmab': [], 'round_robin': [], 'control': []}

# this is paired tuple - [does beneficiary belongs to RMAB, does beneficiary belongs to RR]
arm_to_cat = {'rmab': [1,0], 'round_robin': [0,1], 'control': [0,0]}

# mask defining whether the beneficiary belongs to that group or not. Right now it is all 0,
# It will be updated later in the loop.
mask = {'rmab':[0]*len(pilot_user_ids), 'round_robin':[0]*len(pilot_user_ids), 'control':[0]*len(pilot_user_ids)}
ctr = 0

In [8]:

for idx, user_id in tqdm(enumerate(pilot_user_ids)):

    curr_mat = []
    curr_row = df[df['user_id'] == user_id]

    # get the arm using df
    arm = curr_row['arm'].item()
    
    count_all = 0 # count of engagements - non binarized
    count_bin = 0 # count of engagements - binarized
    count_c = 0 # count of number of calls made

    week0e, week0c = [int(itr) for itr in curr_row['week{}_E/C'.format(0)].item().split('/')]
    count_c += week0c
    if week0e:
        curr_mat.append(1)
        count_all += week0e
        count_bin += 1
    else:
        curr_mat.append(0)
    Y_mat[0].append(count_bin)
    Y_all_mat[0].append(count_all)
    C_mat[0].append(count_c)
    for i in range(1,T):
        nume, numc = [int(itr) for itr in curr_row['week{}_E/C'.format(i)].item().split('/')]
        count_c += numc
        if nume > 0:
            count_all += nume
            count_bin += 1
            curr_mat.append(nume)
        else:
            curr_mat.append(0)
        Y_mat[i].append(count_bin)
        Y_all_mat[i].append(count_all)
        C_mat[i].append(count_c)
    X_mat.append(list(pilot_static_features[idx, :]) + [curr_mat[0]]+ arm_to_cat[arm])

    # update mask based on arm
    mask[arm][ctr] = 1
    ctr+=1

    full_mat[arm].append(np.array(curr_mat, dtype=np.int))

mask = {i:np.array(mask[i][:ctr]) for i in mask}
X_mat = np.array(X_mat)
Y_mat = np.array(Y_mat)
Y_all_mat = np.array(Y_all_mat)
C_mat = np.array(C_mat)


23003it [00:22, 1036.01it/s]


In [39]:
Y_all_mat - Y_all_mat[0]

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 2,  0,  0, ...,  0,  0,  0],
       [ 4,  0,  0, ...,  0,  0,  0],
       ...,
       [18,  3,  3, ...,  0,  0,  4],
       [20,  3,  3, ...,  0,  0,  5],
       [22,  3,  3, ...,  0,  0,  6]])

(23003,)

In [45]:
thresh = CONFIG['thresh']
thresh=0.5

# has the eneficiaries listened to 50% (thresh) of total calls?
Y_thresh_mat = (Y_all_mat > C_mat*thresh).astype(int)
Y_adjusted_thresh_mat = ((Y_all_mat-Y_all_mat[0]) > (C_mat-C_mat[0])*thresh).astype(int)

In [27]:
# load pilot registration data again
reg = pd.read_csv('feb16-mar15_data/beneficiary/ai_registration-20210216-20210315.csv', sep='\t')

interv_df = pd.DataFrame(columns=['user_id', 'intervene_week', 'intervene_date', 'exp_group'])
interv_calling_files = ['250_week1_290421', '400_week2_060521', '400_week3_120521', '400_week4_180521', '435_week5_240521', '600_week6_310521', '700_week7_070621', '1000_week8_140621', '1000_week9_210621', '1000_week10_280621', '1000_week11_050721']

from datetime import datetime
# for every file in interv_calling_file - get its date format - 290421 -> 29/04/21
week_date_lookup = {int(f.split('_')[1].split('week')[-1]):datetime.strptime(f.split('_')[-1], '%d%m%y') for f in interv_calling_files}

for file in interv_calling_files:
    with open('outputs/pilot_generations/calling_list_{}.txt'.format(file), 'r') as fr:
        for line in fr:
            user_id = int(line.strip())
            interv_week = int(file.split('_')[1].split('week')[-1])
            exp_group = df[df['user_id']==user_id]['arm'].iloc[0]
            intervene_date = week_date_lookup[interv_week]
            interv_df = interv_df.append({'user_id': user_id, 'intervene_week': interv_week, 'intervene_date':intervene_date, 'exp_group': exp_group}, ignore_index=True)
interv_df = pd.merge(interv_df, reg[['user_id', 'registration_date']])  
interv_df = pd.merge(interv_df, df[['user_id', 'cluster']])
interv_df['registration_date'] = pd.to_datetime(interv_df['registration_date'])
interv_df['days_since_reg'] = (interv_df['intervene_date'] - interv_df['registration_date']).dt.days

# some of these features arent required but are here nevertheless

In [26]:
# now we need to find whether a beneficiary was intervened by a week t
X_mat_df = pd.DataFrame(X_mat, columns=columns)
interv_X_mat_df = X_mat_df.copy()
interv_X_mat_df['user_id'] = pilot_user_ids # add user_id column
# get first week intervened on for every beneficiary
interv_df['first_interv_week'] = interv_df.groupby('user_id')['intervene_week'].transform(min) 
# note that there could be multiple entries of a beneficiary. Hence lets drop repeated user, first_interv_week pairs
interv_df = interv_df[['user_id', 'first_interv_week']].drop_duplicates()
# now get this variable in our interv_X_mat_df by performing a join
interv_X_mat_df = pd.merge(interv_X_mat_df, interv_df, how='left', on='user_id')
# those beneficiaries who were never intervened on will be NA. So fill them with a large value 
# (they were first intervened at week 999 -> implying they werent intervened)
interv_X_mat_df['first_interv_week'] = interv_X_mat_df['first_interv_week'].fillna(999)

# now add our variable is_intervened_by_week_t as first_interv_week<t. 
# note that it is < and not <=, because interventions at week t doesn't impact engagements at week t
for i in range(1, T):
    interv_X_mat_df[f'is_intervened_by_week{i}'] = (interv_X_mat_df['first_interv_week']<i).astype(int)

KeyError: 'Column not found: intervene_week'

In [30]:
# These are some bad columns that cause issue in fitting logistic regression
# so better drop them

X_mat_df = X_mat_df.drop(columns = [col for col in X_mat_df.columns if col.startswith('phone_owner')]+
                        [col for col in X_mat_df.columns if col.startswith('Channel')]+
                        [col for col in X_mat_df.columns if col.startswith('call_slots')])

In [46]:
# This function removes columns which have very low variance and cause issue
# in logistic regression optimization

from sklearn.feature_selection import VarianceThreshold

def variance_threshold_selector(data, threshold=0.0001):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [54]:
((Y_mat-Y_mat[0])>2).astype(int).mean(axis=1)

array([0.        , 0.        , 0.        , 0.31521975, 0.39542668,
       0.44189888, 0.47554667, 0.50271704, 0.52393166, 0.54327696,
       0.55988349, 0.57449028])

In [49]:

# some helper dicts so as to make code cleaner
mask_cols = ['control', 'round_robin', 'rmab']
dummy_drop_cols = ['exp_arm_rr', 'exp_arm_rr', 'exp_arm_rmab']
dummy_target_cols = ['exp_arm_rmab', 'exp_arm_rmab', 'exp_arm_rr']
exps = ['RMAB vs RR', 'RMAB vs Control', 'RR vs Control']

for t in range(1, T):
    print(f'Significance for T = {t}\n')
    for m, d, tar, e in zip(mask_cols, dummy_drop_cols, dummy_target_cols, exps):
        print(f'\t{e}')
        # make dataframe of our Y matrix. use mask to filter on group A, B only for A vs B comparision
        # note that we use Y_thresh_mat - which will have binary values
        Y_mat_df = pd.DataFrame(((Y_mat[t]-Y_mat[0])>2).astype(int)[t][mask[m]==0],
                                columns = ['output'])
        # For A vs B comparision, column indicating arm C can be dropped
        test_X_mat_df = X_mat_df.drop(columns=d)[mask[m]==0]
        test_X_mat_df = test_X_mat_df.drop(columns='curr_state')
        # add additional variable of is_intervened_by_week_t using interv_X_mat_df defined earlier
#         test_X_mat_df[f'is_intervened_by_week{t}'] = interv_X_mat_df[f'is_intervened_by_week{t}']
        # both X and Y dataframes should have same indices else statsmodel throws error
        test_X_mat_df.index = np.array(Y_mat_df.index)
        # add constant term to the features, and then apply variance selector
        test_X_mat_df = variance_threshold_selector(sm.add_constant(test_X_mat_df), 0.0001)

        # define and fit the model
        # notet that we use Logistic regression to model binary outcome var
        mod = sm.Logit(Y_mat_df, test_X_mat_df)
        fii = mod.fit()
        
        # the output will have significances in the same order as input dataframe's columns
        # tar is the A vs B comparisions dummy indicator
        # look for its position in original columns
        index = list(test_X_mat_df.columns).index(tar)
        # Get data corresponding to our required variable using the index
        # note that we do index + 1 because statsmodel table has header as first entry
        var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[index+1]
        print(f'\t exp: Coef: {coef}, p_val: {p_val}')

        try:
            # do the same process as above to find signficance of is_intervened_by_week_t variable
            index = list(test_X_mat_df.columns).index(f'is_intervened_by_week{t}')
            var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[index+1]
            print(f'\t is_intervened_by_week{t} : Coef: {coef}, p_val: {p_val}')
        except:
            # note that this code is in try except block because for week 1, is_intervened_by_week_1
            # is constant and gets dropped through variance selector function
            pass

#         print(fii.summary().tables[1])
        print('\n')
    print('~'*100)


Significance for T = 1

	RMAB vs RR
         Current function value: inf
         Iterations: 35


/Users/vermashresth/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/vermashresth/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [ ]:
e(t) = r(t) - r(0) -> baseline adjusted weekly eng
c(t) = sigma(e(t)) = sigma(r(s)) - t*r(0)

c(t) = sigma(r(s)) - r(0)

In [24]:

# some helper dicts so as to make code cleaner
mask_cols = ['control', 'round_robin', 'rmab']
dummy_drop_cols = ['exp_arm_rr', 'exp_arm_rr', 'exp_arm_rmab']
dummy_target_cols = ['exp_arm_rmab', 'exp_arm_rmab', 'exp_arm_rr']
exps = ['RMAB vs RR', 'RMAB vs Control', 'RR vs Control']

for t in range(1, T):
    print(f'Significance for T = {t}\n')
    for m, d, tar, e in zip(mask_cols, dummy_drop_cols, dummy_target_cols, exps):
        print(f'\t{e}')
        # make dataframe of our Y matrix. use mask to filter on group A, B only for A vs B comparision
        # note that we use Y_mat, for our original paper's cumulative outcome variable
        Y_mat_df = pd.DataFrame(Y_mat[t][mask[m]==0]-Y_mat[0][mask[m]==0],
                                columns = ['output'])
        # For A vs B comparision, column indicating arm C can be dropped
        test_X_mat_df = X_mat_df.drop(columns=d)[mask[m]==0]
        # add additional variable of is_intervened_by_week_t using interv_X_mat_df defined earlier
        test_X_mat_df[f'is_intervened_by_week{t}'] = interv_X_mat_df[f'is_intervened_by_week{t}']
        # both X and Y dataframes should have same indices else statsmodel throws error
        test_X_mat_df.index = np.array(Y_mat_df.index)
        # add constant term to the features, and then apply variance selector
        test_X_mat_df = variance_threshold_selector(sm.add_constant(test_X_mat_df), 0.0001)

        # define and fit the model
        # note that here we use OLS model for continious outcome var
        mod = sm.OLS(Y_mat_df, test_X_mat_df)
        fii = mod.fit()
        
        # the output will have significances in the same order as input dataframe's columns
        # tar is the A vs B comparisions dummy indicator
        # look for its position in original columns
        index = list(test_X_mat_df.columns).index(tar)
        # Get data corresponding to our required variable using the index
        # note that we do index + 1 because statsmodel table has header as first entry
        var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[index+1]
        print(f'\t exp: Coef: {coef}, p_val: {p_val}')

        try:
            # do the same process as above to find signficance of is_intervened_by_week_t variable
            index = list(test_X_mat_df.columns).index(f'is_intervened_by_week{t}')
            var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[index+1]
            print(f'\t is_intervened_by_week{t} : Coef: {coef}, p_val: {p_val}')
        except:
            # note that this code is in try except block because for week 1, is_intervened_by_week_1
            # is constant and gets dropped through variance selector function
            pass

#         print(fii.summary().tables[1])
        print('\n')
    print('~'*100)


Significance for T = 1

	RMAB vs RR
	 exp: Coef:     0.0094, p_val:  0.135


	RMAB vs Control
	 exp: Coef:     0.0066, p_val:  0.295


	RR vs Control
	 exp: Coef:    -0.0027, p_val:  0.671


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Significance for T = 2

	RMAB vs RR
	 exp: Coef:     0.0225, p_val:  0.036
	 is_intervened_by_week2 : Coef:     0.0675, p_val:  0.113


	RMAB vs Control
	 exp: Coef:     0.0168, p_val:  0.118
	 is_intervened_by_week2 : Coef:     0.0907, p_val:  0.130


	RR vs Control
	 exp: Coef:    -0.0045, p_val:  0.673
	 is_intervened_by_week2 : Coef:     0.0386, p_val:  0.519


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Significance for T = 3

	RMAB vs RR
	 exp: Coef:     0.0269, p_val:  0.075
	 is_intervened_by_week3 : Coef:     0.0464, p_val:  0.218


	RMAB vs Control
	 exp: Coef:     0.0139, p_val:  0.357
	 is_intervened_by_week3 : Coef:     0.0740, p_

In [55]:

# some helper dicts so as to make code cleaner
mask_cols = ['control', 'round_robin', 'rmab']
dummy_drop_cols = ['exp_arm_rr', 'exp_arm_rr', 'exp_arm_rmab']
dummy_target_cols = ['exp_arm_rmab', 'exp_arm_rmab', 'exp_arm_rr']
exps = ['RMAB vs RR', 'RMAB vs Control', 'RR vs Control']

for t in range(1, T):
    print(f'Significance for T = {t}\n')
    for m, d, tar, e in zip(mask_cols, dummy_drop_cols, dummy_target_cols, exps):
        print(f'\t{e}')
        # make dataframe of our Y matrix. use mask to filter on group A, B only for A vs B comparision
        # note that we use Y_mat, for our original paper's cumulative outcome variable
        Y_mat_df = pd.DataFrame(Y_mat[t][mask[m]==0]-(t+1)*Y_mat[0][mask[m]==0],
                                columns = ['output'])
        # For A vs B comparision, column indicating arm C can be dropped
        test_X_mat_df = X_mat_df.drop(columns=d)[mask[m]==0]
        # add additional variable of is_intervened_by_week_t using interv_X_mat_df defined earlier
#         test_X_mat_df[f'is_intervened_by_week{t}'] = interv_X_mat_df[f'is_intervened_by_week{t}']
        # both X and Y dataframes should have same indices else statsmodel throws error
        test_X_mat_df.index = np.array(Y_mat_df.index)
        # add constant term to the features, and then apply variance selector
        test_X_mat_df = variance_threshold_selector(sm.add_constant(test_X_mat_df), 0.0001)

        # define and fit the model
        # note that here we use OLS model for continious outcome var
        mod = sm.OLS(Y_mat_df, test_X_mat_df)
        fii = mod.fit()
        
        # the output will have significances in the same order as input dataframe's columns
        # tar is the A vs B comparisions dummy indicator
        # look for its position in original columns
        index = list(test_X_mat_df.columns).index(tar)
        # Get data corresponding to our required variable using the index
        # note that we do index + 1 because statsmodel table has header as first entry
        var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[index+1]
        print(f'\t exp: Coef: {coef}, p_val: {p_val}')

        try:
            # do the same process as above to find signficance of is_intervened_by_week_t variable
            index = list(test_X_mat_df.columns).index(f'is_intervened_by_week{t}')
            var, coef, std_err, t_val, p_val, q1, q3 = fii.summary().tables[1].data[index+1]
            print(f'\t is_intervened_by_week{t} : Coef: {coef}, p_val: {p_val}')
        except:
            # note that this code is in try except block because for week 1, is_intervened_by_week_1
            # is constant and gets dropped through variance selector function
            pass

#         print(fii.summary().tables[1])
        print('\n')
    print('~'*100)


Significance for T = 1

	RMAB vs RR
	 exp: Coef:     0.0094, p_val:  0.135


	RMAB vs Control
	 exp: Coef:     0.0066, p_val:  0.295


	RR vs Control
	 exp: Coef:    -0.0027, p_val:  0.671


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Significance for T = 2

	RMAB vs RR
	 exp: Coef:     0.0225, p_val:  0.036


	RMAB vs Control
	 exp: Coef:     0.0183, p_val:  0.088


	RR vs Control
	 exp: Coef:    -0.0039, p_val:  0.715


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Significance for T = 3

	RMAB vs RR
	 exp: Coef:     0.0269, p_val:  0.075


	RMAB vs Control
	 exp: Coef:     0.0170, p_val:  0.254


	RR vs Control
	 exp: Coef:    -0.0094, p_val:  0.531


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Significance for T = 4

	RMAB vs RR
	 exp: Coef:     0.0335, p_val:  0.084


	RMAB vs Control
	 exp: Coef:     0.0157, p_val